# Ch05_02.나이브 베이즈(범주형 문제)

## 1.기본 package 설정

In [1]:
## 1.기본
import numpy as np  # numpy 패키지 가져오기
import matplotlib.pyplot as plt # 시각화 패키지 가져오기

## 2.데이터 가져오기
import pandas as pd # csv -> dataframe으로 전환
from sklearn import datasets # python 저장 데이터 가져오기

## 3.데이터 전처리
from sklearn.preprocessing import StandardScaler # 연속변수의 표준화
from sklearn.preprocessing import LabelEncoder # 범주형 변수 수치화

# 4. 훈련/검증용 데이터 분리
from sklearn.model_selection import train_test_split 

## 5.분류모델구축 (3장.p.83~130)
# from sklearn.tree import DecisionTreeClassifier # 결정 트리
from sklearn.naive_bayes import GaussianNB # 나이브 베이지안 (실수형)
from sklearn.naive_bayes import MultinomialNB # 나이브 베이지안(정수형)
from sklearn.naive_bayes import BernoulliNB # 나이브 베이지안(바이너리형(0,1))
# from sklearn.neighbors import KNeighborsClassifier # K-최근접 이웃
# from sklearn.ensemble import RandomForestClassifier # 랜덤 포레스트
# from sklearn.ensemble import BaggingClassifier # 앙상블
# from sklearn.linear_model import Perceptron # 퍼셉트론
# from sklearn.linear_model import LogisticRegression # 로지스틱 회귀 모델
# from sklearn.svm import SVC # 서포트 벡터 머신(SVM)
# from sklearn.neural_network import MLPClassifier # 다층인공신경망

## 6.모델검정
from sklearn.metrics import confusion_matrix, classification_report # 정오분류표
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score # 정확도, 민감도 등
from sklearn.metrics import roc_curve, auc # ROC 곡선 그리기

## 7.최적화
from sklearn.model_selection import cross_validate # 교차타당도
from sklearn.pipeline import make_pipeline # 파이프라인 구축
from sklearn.model_selection import learning_curve, validation_curve # 학습곡선, 검증곡선
from sklearn.model_selection import GridSearchCV # 하이퍼파라미터 튜닝

## 2.데이터 가져오기

### 2.1 데이터프레임으로 저장
- 원본데이터(csv)를 dataframe 형태로 가져오기(pandas)

In [2]:
delays_df = pd.read_csv('FlightDelays.csv')
delays_df.head()

,CRS_DEP_TIME,CARRIER,DEP_TIME,DEST,DISTANCE,FL_DATE,FL_NUM,ORIGIN,Weather,DAY_WEEK,DAY_OF_MONTH,TAIL_NUM,FlightStatus
0,1455,OH,1455,JFK,184,01/01/2004,5935,BWI,0,THU,1,N940CA,ontime
1,1640,DH,1640,JFK,213,01/01/2004,6155,DCA,0,THU,1,N405FJ,ontime
2,1245,DH,1245,LGA,229,01/01/2004,7208,IAD,0,THU,1,N695BR,ontime
3,1715,DH,1709,LGA,229,01/01/2004,7215,IAD,0,THU,1,N662BR,ontime
4,1039,DH,1035,LGA,229,01/01/2004,7792,IAD,0,THU,1,N698BR,ontime


- 자료구조 살펴보기

In [3]:
delays_df.shape

(2201, 13)

In [4]:
# 자료구조 살펴보기
delays_df.keys()

Index(['CRS_DEP_TIME', 'CARRIER', 'DEP_TIME', 'DEST', 'DISTANCE', 'FL_DATE',
       'FL_NUM', 'ORIGIN', 'Weather', 'DAY_WEEK', 'DAY_OF_MONTH', 'TAIL_NUM',
       'FlightStatus'],
      dtype='object')

### 2.2 data와 target으로 분리
- 필요한 데이터만 추출
- data: X, target: y 로 분리

In [5]:
X=delays_df.iloc[:,[9,0,7,3,1]]
X.head()

,DAY_WEEK,CRS_DEP_TIME,ORIGIN,DEST,CARRIER
0,THU,1455,BWI,JFK,OH
1,THU,1640,DCA,JFK,DH
2,THU,1245,IAD,LGA,DH
3,THU,1715,IAD,LGA,DH
4,THU,1039,IAD,LGA,DH


In [6]:
y=delays_df.iloc[:,[12]]
y.head()

,FlightStatus
0,ontime
1,ontime
2,ontime
3,ontime
4,ontime


## 3.데이터 전처리

### 3.1 data(X) 레이블 인코딩
- 문자형 자료를 숫자(범주형)로 인코딩
- 숫자형 자료를 표준화
- 결정나무, 랜덤 포레스트, 나이브 베이즈 분류 : 원본데이터 그대로 유지

- 출발예정시간변환(18):6 = 6 AM ~ 21 = 21

In [7]:
X['CRS_DEP_TIME']=round(X['CRS_DEP_TIME']/100)

C:\Users\leecho\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
X.head()

,DAY_WEEK,CRS_DEP_TIME,ORIGIN,DEST,CARRIER
0,THU,15.0,BWI,JFK,OH
1,THU,16.0,DCA,JFK,DH
2,THU,12.0,IAD,LGA,DH
3,THU,17.0,IAD,LGA,DH
4,THU,10.0,IAD,LGA,DH


- 출발예정시간을 범주형 변수로 변환

In [9]:
class_le = LabelEncoder()

X.iloc[:, [0]] = class_le.fit_transform(X.iloc[:,[0]])
X.iloc[:, [2]] = class_le.fit_transform(X.iloc[:,[2]])
X.iloc[:, [3]] = class_le.fit_transform(X.iloc[:,[3]])
X.iloc[:, [4]] = class_le.fit_transform(X.iloc[:,[4]])

C:\Users\leecho\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\leecho\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [10]:
X.head()

,DAY_WEEK,CRS_DEP_TIME,ORIGIN,DEST,CARRIER
0,4,15.0,0,1,4
1,4,16.0,1,1,1
2,4,12.0,2,2,1
3,4,17.0,2,2,1
4,4,10.0,2,2,1


### 3.2 Class(target) 레이블 인코딩
- class(y)가 문자형으로 숫자로 변환
- 연착을 중요변수로 설정 
- 정상(1), 연착(0) --> 정상(1), 연착(0)

In [11]:
y = y.replace (['ontime','delayed'], [0,1])
y.head()

,FlightStatus
0,0
1,0
2,0
3,0
4,0


- 분석을 위해서는 데이터 프레임 형태를 배열형태로 변경해야 함

In [12]:
class_le = LabelEncoder()

y = class_le.fit_transform(y)
y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

## 4.훈련/검증용 데이터 분할
- test_size: 검증데이터 30%
- random_state : random seed 번호 = 1
- stratify : y(클래스)의 비율에 따라 분할

In [13]:
# \ 이후에 space 없어야 함
X_train, X_test, y_train, y_test = \
        train_test_split(X, y, 
                         test_size=0.3, 
                         random_state=1, 
                         stratify=y)

## 5.모델구축
- 실수형: GaussianNB (0.1, 0.9, 1.5)
- 정수형: MultinomialNB (1, 3, 2)
- 바이너리형: BernoulliNB (0,1,0)
- alpha : smoothing

In [14]:
mnb = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
mnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

## 6. 모델검정

### 6.1 검증용 데이터로 예측
- predict (class로 표시)
- predict_proba (확률값으로 표시)

In [15]:
y_pred = mnb.predict(X_test)

### 6.2 정오분류표로 검정

In [16]:
confmat = pd.DataFrame(confusion_matrix(y_test, y_pred),
                      index=['True[0]','True[1]'],
                      columns=['Predict[0]', 'Predict[1]'])
confmat

,Predict[0],Predict[1]
True[0],532,0
True[1],129,0


In [17]:
print('Classification Report')
print(classification_report(y_test, y_pred))

Classification Report
              precision    recall  f1-score   support

           0       0.80      1.00      0.89       532
           1       0.00      0.00      0.00       129

    accuracy                           0.80       661
   macro avg       0.40      0.50      0.45       661
weighted avg       0.65      0.80      0.72       661



C:\Users\leecho\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6.3 정확도, 민감도 확인
- 클래스가 2개일 경우에만 실행

In [18]:
print('잘못 분류된 샘플 개수: %d' % (y_test != y_pred).sum())
print('정확도: %.3f' % accuracy_score(y_test, y_pred))
print('정밀도: %.3f' % precision_score(y_true=y_test, y_pred=y_pred))
print('재현율: %.3f' % recall_score(y_true=y_test, y_pred=y_pred))
print('F1: %.3f' % f1_score(y_true=y_test, y_pred=y_pred))

잘못 분류된 샘플 개수: 129
정확도: 0.805
정밀도: 0.000
재현율: 0.000
F1: 0.000


C:\Users\leecho\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\leecho\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6.4 ROC 곡선 그리기
- decision_function 사용가능한 모델일 경우: tree.decision_function(X_test)
- decision_function 사용이 안되는 모델일 경우:tree.predict_proba(X_test)[:, 1]